# Şifreleme

Klasik şifreleme 2 kısma ayrılır: Simetrik şifreleme ve asimetrik şifreleme.
Simetrik şifreleme, tarihsel olarak geçmişi de bulunan bir şifreleme yöntemidir. 
Bu yöntemde şifreli ve çözülmüş mesaj aynı gizli anahtarla bulunur.
Tek kullanımlık anahtar şifreleri gibi bazı şifreler, bilgisayar korsanlarının keyfi
saldırılarına karşı mükemmel bir koruma sağlar.

## Simetrik Şifreleme (Gizli Anahtarlı Şifreleme)

Gizli anahtarlı şifreleme yönteminde, ortada bir adet anahtar bulunur. Bu anahtar açık metni şifreler ve şifreli 
metni açık metine çevirir. Burada önemli olan anahtarın güvenliğidir. Eğer gizli anahtar başkalarının eline 
geçerse tüm şifrelenmiş metinler kolaylıkla çözebilir.

Simetrik şifrelemenin avantajları şunlardır:
* Simetrik şifreleme oldukça hızlıdır
* Donanımla beraber kullanılabilirler
* Anahtarları oldukça kısadır
* Daha güçlü şifrelerin oluşturulmasında aracı olarak kullanılabilirler
* Kullanımları gayet anlaşılır ve kolaydır

Simetrik şifrelemenin dezavantajları şunlardır:
* Anahtarın saklanması ve taraflara ulaştırılması zordur, güvenlik riski içerir
* Anahtarların tek defa kullanılması gerekmektedir, dolayısıyla ölçeklenmesi zordur.
* Kimlik doğrulama işlevi görülmez. Anahtara sahip olan herkes olaya dahil olur.
* Bütünlük sağlaması yoktur. Anahtarı ele geçiren kişi veriyi değiştirmiş olabilir.
* İnkar edilememezlik sağlamaz.

# BB84

C. H. Bennett ve G. Brassard tarafından 1984 yılında getirilen bu protokol,
kuantum süperpozisyonların ancak gözlem anında bir duruma çökmesi prensibine dayanır.

Rastgele bir durumda gönderilen tekil kubitler, karşıdaki kullanıcı (Bora)
tarafından ölçülür, ardından ölçüm yolları kıyaslanır.

Ölçüm yollarının uyuştuğu durumda, ölçümler de uyuşacaktır. Bu, iki tarafın da elinde
eşit, rastgele bir bit dizisi olması anlamına gelir.

## Güvenlik

Kuantum iletişim kanalında istenmeyen bir dinleyici olması durumunda,
"dinle ve tekrar gönder" saldırısı da olasılıksal bir doğa edinir:
Dinleyici ve asıl alıcının uyuşması, yani dinleyici varlığının gizli kalması,
kubit başına 1/4 ihtimallidir. Kubit sayısı arttıkça dinleyicinin kusursuz
gizliliği ihtimali 0'a yaklaşır.

In [123]:
from random import getrandbits
from time import sleep
from qiskit import QuantumCircuit, execute, Aer

def grb(n):
    """n rastgele bit'lik bir dizi (string) döner."""
    r = bin(getrandbits(n))[2:]
    if len(r)<n:
        r = '0'*(n-len(r)) + r
    return r

def polarizer(w:str)->str:
    """1 ve 0'lardan oluşan bir bit dizisini + ve x polarizasyona döndürür."""
    s = ''
    for i in w:
        if i == '0':
            s += '+'
        elif i == '1':
            s += 'x'
    return s

In [124]:
def BB84(a_bit, a_direction, b_direction):
    """
    BB84 protokolünü, tek bit'lik sistem için çalıştırır,
    öyle ki a_direction ve b_direction'ın uyuştuğu durumda,
    a_bit ve sonuç bit'i de uyuşacaktır.
    
    BB84 seri çalışan bir protokol olduğu için,
    n-bit'lik sistem bunun n kere tekrarıyla gerçekleşecektir.
    
    Tüm girdiler ve çıktı, '0' veya '1' olacaktır.
    """    
    circ = QuantumCircuit(1,1)
    
    if(a_bit=='1'):
        circ.x(0)
    if(a_direction=='1'):
        circ.h(0)
    if(b_direction=='1'):
        circ.h(0)
    circ.measure(0,0)
    
    return execute(circ, Aer.get_backend("qasm_simulator"), shots=1, memory=True).result().get_memory()[0]

def BB84_DEMO(t):
    target = 2*t

    print("\t\tAYŞE\t\t\t\t\t\tBORA\n"+'='*88)
    # Ayşe ve Bora için 44'er karakter sınırı
    
    print("- Arkadaşım Bora'ya göndereceğim mesajın\n  güvenli olması için tek kullanımlık")
    print("  bir anahtar göndermek istiyorum.\n  Bunun için rastgele pozisyonda\n  fotonlar kullanacağım.")
    
    alice_bits = grb(target)
    alice_directions = grb(target)
    sleep(2)
    print("\n- Elimdeki rastgele bit'ler:")
    i = 0
    while(i*32<len(alice_bits)):
        print('    '+alice_bits[32*i:min(32*(i+1),len(alice_bits))])
        i += 1
    print("\n- Elimdeki rastgele yönler:")
    aqs = polarizer(alice_directions)
    i = 0
    while(i*32<len(aqs)):
        print('    '+aqs[32*i:min(32*(i+1),len(aqs))])
        i += 1
    
    bob_directions = grb(target)
    bob_bits = ''
    
    sleep(2)
    q_comm = '|'
    # Ayşe'nin rastgele bit ve yönleri var,
    # Bora'nın rastgele yönleri var ve buna göre ölçüm yapar.
    for i in range(target):
        pair = alice_bits[i]+alice_directions[i]
        if(pair=='00'):
            q_comm += '0'
        elif(pair=='10'):
            q_comm += '1'
        elif(pair=='01'):
            q_comm += '+'
        elif(pair=='11'):
            q_comm += '-'
        bob_bits += BB84(*pair, bob_directions[i])    
    q_comm += '>'
    
    print("\n- Bora'ya gönderilen kuantum durum:\n  ",end='')
    for i in range(8):
        print('-', end='')
        sleep(0.15)
    print('  ', end='')
    for i in q_comm:
        print(i, end='')
        sleep(0.05)
    print('  ', end='')
    for i in range(7):
        print('-', end='')
        sleep(0.15)
    print('>\n')
    
    c2 = '\t'*5+' '*4 # yani 44 boş karakter, bizi 2. sütuna taşıyor
    sleep(2)
    print(c2+"- Ayşe'nin mesajıyla aynı boyda\n"+c2+"  rastgele yön belirliyorum:")
    bqs = polarizer(bob_directions)
    i = 0
    while(i*32<len(bqs)):
        print(c2+'    '+bqs[32*i:min(32*(i+1),len(bqs))])
        i += 1
        
    sleep(2)
    print('\n'+c2+"- Bu rastgele yöne göre ölçümlerim:")
    i = 0
    while(i*32<len(bob_bits)):
        print(c2+'    '+bob_bits[32*i:min(32*(i+1),len(bob_bits))])
        i += 1
        
    sleep(2)
    print('\n\t\tRastgele yönlerimizi ortak kanalda karşılaştırıyoruz:')
    print('  -Ayşe:\t' + aqs)
    sleep(0.5)
    print('\t\t' + bqs + '\t:Bora-')
    print('\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('=', end='')
            sleep(0.1)
        else:
            print(' ', end='')
            
    sleep(2)
    print('\n\t\tAynı yönlerde bit\'ler de aynı olacaktır:\n\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('\u2193', end='')
            sleep(0.05)
        else:
            print(' ', end='')
    print('\n  -Ayşe:\t' + alice_bits)
    print('\t\t' + bob_bits + '\t:Bora-\n\t\t', end='')
    for i in zip(alice_directions, bob_directions):
        if(i[0]==i[1]):
            print('\u2193', end='')
            sleep(0.05)
        else:
            print(' ', end='')
    print('\n\t\t', end='')
    count = 0
    for i in zip(alice_directions, bob_directions, alice_bits, bob_bits):
        if(i[0]==i[1]):
            print(i[2], end='')
            sleep(0.15)
            count += 1
        else:
            print(' ', end='')
            
    print('\n\n- Bunun sonunda ikimizin de elinde,\n'+'  {} bit\'lik bir dizi oldu.'.format(count))
    print('  (Hedefimiz ise {} bit\'lik bir diziydi.)\n'.format(t))
    sleep(1)
    print(c2+'- Eğer kusur veya dinlenme riski varsa,')
    print(c2+'  bunun belli bir kısmı karşılaştırılabilir.\n')
    sleep(1)
    print('- Uyuşmazlık belli bir oranın üzerindeyse,\n  bunu kullanmak yerine baştan başlarız.')

    # Demo'nun geliştirilmesi için,
    #   * kıyas adımı gösterilebilir
    #   * yeni protokoller eklenebilir

In [125]:
# Görselleştirme açısından HEDEF bit sayısının 32 olarak tutulması önerilmektedir.
HEDEF = 32

BB84_DEMO(HEDEF)

		AYŞE						BORA
- Arkadaşım Bora'ya göndereceğim mesajın
  güvenli olması için tek kullanımlık
  bir anahtar göndermek istiyorum.
  Bunun için rastgele pozisyonda
  fotonlar kullanacağım.

- Elimdeki rastgele bit'ler:
    01100010111111000010000000111011
    10000100111110110000100111100101

- Elimdeki rastgele yönler:
    x++x++xx+++xxx++xxx++xxx++x+++x+
    ++xx+++x++x++++xxxx+++xxx++x+x+x

- Bora'ya gönderilen kuantum durum:
  --------  |+11+00-+111---00++-00+++00-110-110++010+11-1101-+++010+--11+0-0->  ------->

					    - Ayşe'nin mesajıyla aynı boyda
					      rastgele yön belirliyorum:
					        +++x+x+x+++xxx+++xxx+xxxx+x+++x+
					        x++++++x+x++++xx+x+xxxxx++++xx+x

					    - Bu rastgele yöne göre ölçümlerim:
					        01100000111111000011000000111011
					        00110100110110010000110111101101

		Rastgele yönlerimizi ortak kanalda karşılaştırıyoruz:
  -Ayşe:	x++x++xx+++xxx++xxx++xxx++x+++x+++xx+++x++x++++xxxx+++xxx++x+x+x
		+++x+x+x+++xxx+++xxx+xxxx+x+++x+x++